<a href="https://colab.research.google.com/github/cleysonl/deepul/blob/deepul-branch-2/Pytorch_aux_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [0]:
x = torch.zeros(100)
logits = nn.Parameter(x, requires_grad=True)

In [3]:
# x.shape
logits.shape

torch.Size([100])

In [4]:
logits.unsqueeze(0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]], grad_fn=<UnsqueezeBackward0>)

In [5]:
logits

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], requires_grad=True)

In [6]:
logits.unsqueeze(0).repeat(x.shape[0],1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<RepeatBackward>)

In [7]:
logits.unsqueeze(0).repeat(x.shape[0],1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<RepeatBackward>)

In [0]:
y = F.softmax(logits, dim=0).unsqueeze(1).repeat(1, 1000 // 100).view(-1)

In [9]:
print(y.shape)

torch.Size([1000])


In [0]:
means = nn.Parameter(torch.arange(4).float() / (4 - 1) * 100, requires_grad=True)

In [11]:
means.shape

torch.Size([4])

In [12]:
print(means)

Parameter containing:
tensor([  0.0000,  33.3333,  66.6667, 100.0000], requires_grad=True)


In [13]:
means.unsqueeze(0)

tensor([[  0.0000,  33.3333,  66.6667, 100.0000]],
       grad_fn=<UnsqueezeBackward0>)

In [0]:
log_scales = nn.Parameter(torch.randn(4), requires_grad=True)

In [15]:
inv_scales = torch.exp(-log_scales)
inv_scales

tensor([0.5661, 0.4696, 1.9795, 3.7270], grad_fn=<ExpBackward>)

In [0]:
x = 2.
plus_in = inv_scales*(x+0.5)
min_in = inv_scales*(x-0.5)
cdf_plus = torch.sigmoid(plus_in)
cdf_min = torch.sigmoid(min_in)

In [17]:
print(cdf_plus)
print(cdf_min)

tensor([0.8046, 0.7639, 0.9930, 0.9999], grad_fn=<SigmoidBackward>)
tensor([0.7004, 0.6692, 0.9512, 0.9963], grad_fn=<SigmoidBackward>)


In [18]:
1/(1+torch.exp(-plus_in))

tensor([0.8046, 0.7639, 0.9930, 0.9999], grad_fn=<MulBackward0>)

In [19]:
1/(1+torch.exp(-min_in))

tensor([0.7004, 0.6692, 0.9512, 0.9963], grad_fn=<MulBackward0>)

In [0]:
cdf_delta = cdf_plus - cdf_min

In [21]:
cdf_delta

tensor([0.1042, 0.0947, 0.0418, 0.0036], grad_fn=<SubBackward0>)

In [22]:
 log_cdf_delta = torch.log(torch.clamp(cdf_delta, min=1e-12))
 print(log_cdf_delta)

tensor([-2.2614, -2.3569, -3.1750, -5.6187], grad_fn=<LogBackward>)


In [0]:
#  log_cdf_delta = torch.log(cdf_delta)
#  print(log_cdf_delta)

In [24]:
log_cdf_plus = torch.log(torch.clamp(torch.sigmoid(inv_scales*(0.5 - means)), min=1e-12)) # edge case: x=0 
log_cdf_min = torch.log(torch.clamp(1- torch.sigmoid(inv_scales*(100 - 1.5 - means)), min=1e-12)) # edge case: x=d-1
print(log_cdf_min)
print(log_cdf_plus)

tensor([-2.7631e+01, -2.7631e+01, -2.7631e+01, -3.7264e-03],
       grad_fn=<LogBackward>)
tensor([ -0.5616, -15.4198, -27.6310, -27.6310], grad_fn=<LogBackward>)


In [0]:
# log_cdf_plus = torch.log(torch.sigmoid(inv_scales*(0.5 - means))) # edge case: x=0 
# log_cdf_min = torch.log(1- torch.sigmoid(inv_scales*(100 - 1.5 - means))) # edge case: x=d-1
# print(log_cdf_min)
# print(log_cdf_plus)

## **MADE auxiliar**

In [0]:
net = []
nin = 6
nout = 6*100
hidden_size = [512,512,512]
hs = [nin] + hidden_size + [nout]

In [29]:
for h0, h1 in zip(hs, hs[1:]):
  print(h0, h1)
  net.extend([[h0,h1],0])

6 512
512 512
512 512
512 600


In [31]:
len(hidden_size)

3

In [0]:
m ={}
m[-1]= np.arange(nin) 
for l in range(len(hidden_size)):
  m[l]=np.random.randint(m[l-1].min(), nin-1, size = hidden_size[l])

In [37]:
m

{-1: array([0, 1, 2, 3, 4, 5]),
 0: array([2, 3, 2, 2, 2, 2, 2, 3, 2, 3, 4, 0, 4, 2, 4, 0, 0, 1, 4, 1, 4, 0,
        3, 1, 3, 2, 0, 1, 0, 2, 3, 2, 2, 0, 3, 3, 3, 4, 2, 1, 2, 0, 3, 4,
        4, 0, 4, 4, 2, 2, 3, 1, 4, 4, 1, 2, 4, 2, 1, 4, 0, 2, 2, 3, 1, 2,
        2, 0, 3, 3, 0, 3, 4, 3, 3, 2, 1, 1, 3, 3, 3, 4, 4, 1, 0, 2, 1, 4,
        1, 0, 4, 0, 2, 0, 1, 4, 4, 0, 2, 1, 2, 0, 0, 1, 4, 0, 4, 4, 2, 0,
        3, 2, 3, 2, 2, 1, 3, 4, 4, 1, 4, 4, 0, 4, 2, 4, 0, 2, 4, 2, 3, 2,
        3, 4, 1, 0, 2, 0, 3, 1, 3, 0, 4, 2, 4, 1, 0, 2, 2, 3, 4, 3, 1, 4,
        1, 1, 0, 1, 1, 3, 0, 3, 0, 2, 0, 3, 4, 4, 1, 3, 2, 1, 0, 0, 4, 4,
        1, 3, 4, 1, 3, 2, 3, 2, 0, 1, 1, 0, 1, 2, 4, 3, 0, 0, 1, 2, 2, 1,
        3, 3, 2, 3, 3, 1, 0, 4, 2, 3, 3, 0, 4, 3, 1, 3, 1, 3, 3, 3, 4, 3,
        1, 2, 4, 0, 3, 1, 2, 4, 3, 2, 3, 2, 1, 2, 3, 0, 2, 3, 2, 2, 3, 2,
        3, 4, 4, 2, 0, 1, 3, 0, 0, 1, 2, 4, 0, 4, 2, 2, 3, 0, 0, 2, 2, 4,
        4, 4, 0, 2, 1, 0, 2, 4, 1, 0, 2, 4, 4, 2, 1, 4, 2, 2, 3, 1, 0, 1,
   

In [0]:
masks = [m[l -1][:, None] <= m[l][None, :] for l in range(len(hidden_size))]

In [41]:
m[1][None, :]

array([[4, 4, 3, 3, 3, 2, 2, 3, 1, 0, 3, 3, 4, 0, 0, 2, 3, 4, 2, 2, 0, 3,
        1, 2, 0, 4, 4, 3, 2, 1, 3, 3, 1, 2, 2, 0, 3, 1, 2, 2, 0, 2, 1, 0,
        3, 3, 4, 2, 1, 1, 1, 4, 0, 3, 2, 1, 1, 1, 0, 3, 3, 1, 2, 3, 3, 2,
        1, 4, 2, 3, 2, 3, 0, 0, 4, 0, 0, 0, 1, 0, 3, 4, 3, 1, 1, 0, 2, 4,
        4, 3, 4, 1, 2, 4, 3, 4, 2, 3, 1, 4, 3, 2, 2, 4, 2, 3, 1, 3, 0, 0,
        0, 3, 0, 1, 1, 2, 4, 2, 1, 2, 2, 2, 2, 1, 4, 0, 3, 4, 3, 4, 1, 1,
        4, 1, 2, 0, 1, 1, 4, 4, 0, 3, 3, 0, 1, 0, 0, 1, 0, 2, 1, 1, 4, 2,
        4, 0, 3, 0, 3, 2, 4, 1, 3, 4, 1, 1, 1, 3, 2, 0, 1, 1, 3, 4, 2, 0,
        3, 1, 3, 2, 4, 3, 0, 4, 3, 3, 4, 0, 3, 3, 0, 2, 2, 2, 1, 2, 2, 4,
        2, 4, 1, 0, 3, 3, 1, 4, 0, 2, 4, 2, 2, 1, 3, 0, 4, 1, 1, 3, 2, 3,
        4, 0, 4, 3, 0, 1, 0, 1, 4, 2, 1, 0, 4, 2, 3, 1, 1, 2, 2, 3, 3, 0,
        4, 2, 3, 0, 2, 1, 1, 2, 2, 0, 4, 4, 2, 1, 0, 2, 4, 4, 3, 1, 1, 4,
        0, 2, 1, 0, 4, 0, 1, 3, 0, 0, 0, 3, 2, 4, 4, 1, 2, 2, 3, 2, 0, 1,
        0, 4, 4, 3, 0, 4, 4, 0, 1, 0, 

In [0]:
masks.append(m[3-1][:,None]< m[-1][None,:])

In [43]:
masks

[array([[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [False,  True, False, ...,  True,  True, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([[ True,  True,  True, ..., False, False,  True],
        [ True,  True,  True, ..., False, False,  True],
        [ True,  True,  True, ..., False, False,  True],
        ...,
        [ True,  True,  True, ..., False, False,  True],
        [ True,  True,  True, ..., False, False,  True],
        [ True,  True,  True, ..., False, False,  True]]),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
 

In [0]:
masks[-1] = np.repeat(masks[-1], 100, axis=1)

In [46]:
masks[-1]
masks[-1].shape

(512, 600)

In [47]:
masks[0].shape

(6, 512)